In [3]:
import numpy as np
import os
import sys
import datetime

import scipy as sp
import scipy.stats

from bisect import bisect
from statsmodels.stats.multitest import multipletests

import math

In [4]:
def parse_cfgfile(cfg_file):
    '''Parse configure file
    '''
    Group1_Tophat_aligned_file=''
    Group2_Tophat_aligned_file=''
    output_directory=''
    Annotated_3UTR_file=''
    Output_result_file=''
    Num_least_in_group1_local=''
    Num_least_in_group2_local=''
    Coverage_cutoff_local = ''
    FDR_cutoff_local = ''
    Fold_change_cutoff_local = ''
    PDUI_cutoff_local = ''
    
    for line in open(cfg_file,'r'):
        if line[0] == '\n' or line[0] == '#':
            comments = line;
        else:
            line = line.rstrip();
            command = line.split('=');
            if command[0] == 'Group1_Tophat_aligned_Wig':
                Group1_Tophat_aligned_file = command[1].split(',');
            if command[0] == 'Group2_Tophat_aligned_Wig':
                Group2_Tophat_aligned_file = command[1].split(',');
            if command[0] == 'Output_directory':
                output_directory = command[1]
                if output_directory[-1] != '/':
                    output_directory += '/'
            if command[0] == 'Annotated_3UTR':
                Annotated_3UTR_file = command[1]
            if command[0] == 'Output_result_file':
                Output_result_file = command[1]
            
            ##Parameters
            if command[0] == 'Num_least_in_group1':
                Num_least_in_group1_local = command[1]
            if command[0] == 'Num_least_in_group2':
                Num_least_in_group2_local = command[1]
            if command[0] == 'Coverage_cutoff':
                Coverage_cutoff_local = command[1]
            if command[0] == 'FDR_cutoff':
                FDR_cutoff_local = command[1]
            if command[0] == 'Fold_change_cutoff':
                Fold_change_cutoff_local = command[1]
            if command[0] == 'PDUI_cutoff':
                PDUI_cutoff_local = command[1]
            
    
    if Group1_Tophat_aligned_file=='':
        print("No Tophat aligned BAM file for group 1!", file=sys.stderr)
        exit(1)
    if Group2_Tophat_aligned_file=='':
        print("No Tophat aligned BAM file for group 2!", file=sys.stderr)
        exit(1)
    if output_directory=='':
        print("No output directory!", file=sys.stderr)
        exit(1)
    if Annotated_3UTR_file=='':
        print("No annotated 3' UTR file!", file=sys.stderr)
        exit(1)
    if Output_result_file=='':
        print("No result file name!", file=sys.stderr)
        exit(1)
    return Group1_Tophat_aligned_file,Group2_Tophat_aligned_file,output_directory,Annotated_3UTR_file,Output_result_file,Num_least_in_group1_local,Num_least_in_group2_local,Coverage_cutoff_local,FDR_cutoff_local,Fold_change_cutoff_local,PDUI_cutoff_local


In [5]:
Group1_Tophat_aligned_file,Group2_Tophat_aligned_file,output_directory,Annotated_3UTR_file,Output_result_file,Num_least_in_group1_local,Num_least_in_group2_local,Coverage_cutoff_local,FDR_cutoff_local,Fold_change_cutoff_local,PDUI_cutoff_local = parse_cfgfile('/home/li/桌面/PROJECT6/apasite_predict2/DAPARS/DATA/DaPars_test_data_configure.txt')

In [19]:
output_directory='DaPars_Test_data/'

In [7]:
num_group_1 = len(Group1_Tophat_aligned_file)
All_Sample_files = Group1_Tophat_aligned_file[:]
All_Sample_files.extend(Group2_Tophat_aligned_file)

In [8]:
def Load_Target_Wig_files(All_Wig_files, UTR_Annotation_file):
    UTR_events_dict = {}
    All_Samples_Total_depth = []
    for line in open(UTR_Annotation_file,'r'):
        fields = line.strip('\n').split('\t')
        curr_chr = fields[0]
        region_start = int(float(fields[1]))
        region_end   = int(float(fields[2]))
        curr_strand  = fields[-1]
        UTR_pos = "%s:%s-%s" % (curr_chr, region_start, region_end)
        end_shift = int(round(abs(int(region_start) - int(region_end)) * 0.2))
        if curr_strand == '+':
            region_end = str(int(region_end) - end_shift)
        else:
            region_start = str(int(region_start) + end_shift)
        region_start = int(region_start) + 1
        region_end   = int(region_end) - 1
        if region_start + 50 < region_end:
            UTR_events_dict[fields[3]] = [fields[0],region_start,region_end,fields[-1],UTR_pos]

    ##Load coverage for all samples
    All_samples_extracted_3UTR_coverage_dict = {}
    for curr_wig_file in All_Wig_files:
        curr_sample_All_chroms_coverage_dict = {}
        num_line = 0
        cur_sample_total_depth = 0
        for line in open(curr_wig_file,'r'):
            if '#' not in line and line[0:3] == 'chr':
                fields = line.strip('\n').split('\t')
                chrom_name = fields[0]
                region_start = int(float(fields[1]))
                region_end = int(float(fields[2]))
                cur_sample_total_depth += int(float(fields[-1])) * (region_end - region_start)
                if chrom_name not in curr_sample_All_chroms_coverage_dict:
                    curr_sample_All_chroms_coverage_dict[chrom_name] = [[0],[0]]
                if region_start > curr_sample_All_chroms_coverage_dict[chrom_name][0][-1]:
                    curr_sample_All_chroms_coverage_dict[chrom_name][0].append(region_start)
                    curr_sample_All_chroms_coverage_dict[chrom_name][1].append(0)
                curr_sample_All_chroms_coverage_dict[chrom_name][0].append(region_end)
                curr_sample_All_chroms_coverage_dict[chrom_name][1].append(int(float(fields[-1])))
            num_line += 1
        curr_sample_All_chroms_coverage_dict[chrom_name][1].append(0)
        All_Samples_Total_depth.append(cur_sample_total_depth)
        for curr_3UTR_event_id in UTR_events_dict:
            curr_3UTR_structure = UTR_events_dict[curr_3UTR_event_id]
            curr_chr = curr_3UTR_structure[0]
            if curr_chr in curr_sample_All_chroms_coverage_dict:
                curr_chr_coverage = curr_sample_All_chroms_coverage_dict[curr_chr]
                region_start = curr_3UTR_structure[1]
                region_end = curr_3UTR_structure[2]
                left_region_index = bisect(curr_chr_coverage[0],region_start)
                right_region_index = bisect(curr_chr_coverage[0],region_end)

                extracted_coverage = curr_chr_coverage[1][left_region_index:right_region_index+1]
                extracted_3UTR_region = curr_chr_coverage[0][left_region_index:right_region_index]
                extracted_3UTR_region.insert(0,region_start)
                extracted_3UTR_region.append(region_end)
                if curr_3UTR_event_id not in All_samples_extracted_3UTR_coverage_dict:
                    All_samples_extracted_3UTR_coverage_dict[curr_3UTR_event_id] = []
                All_samples_extracted_3UTR_coverage_dict[curr_3UTR_event_id].append([extracted_coverage,extracted_3UTR_region])
    return All_samples_extracted_3UTR_coverage_dict,np.array(All_Samples_Total_depth),UTR_events_dict


In [9]:
#Load_Target_Wig_files
All_samples_Target_3UTR_coverages, All_samples_sequencing_depths, UTR_events_dict = Load_Target_Wig_files(All_Sample_files, Annotated_3UTR_file)

In [10]:
#All_samples_Target_3UTR_coverages
len(All_samples_Target_3UTR_coverages)

1043

In [13]:
#All_samples_sequencing_depths
All_samples_sequencing_depths

array([475249457, 453111546])

In [14]:
#UTR_events_dict
len(UTR_events_dict)

24983

In [31]:
for i in UTR_events_dict:
    print(i)

NM_001003805|ATP5S|chr14|+
NM_001003800|BICD2|chr9|-
NM_001008781|FAT3|chr11|+
NM_001008783|SLC35D3|chr6|+
NR_003227|AFG3L1P|chr16|+
NR_003226|AFG3L1P|chr16|+
NR_003225|LGALS3|chr14|+
NR_003224|SNORD114-31|chr14|+
NR_003223|SNORD114-30|chr14|+
NR_003222|SNORD114-29|chr14|+
NR_003221|SNORD114-28|chr14|+
NR_003220|SNORD114-27|chr14|+
NR_003229|SNORD113-1|chr14|+
NR_003228|AFG3L1P|chr16|+
NM_001468|GAGE1|chrX|+
NM_001469|XRCC6|chr22|+
NM_001466|FZD2|chr17|+
NM_001467|SLC37A4|chr11|-
NM_001464|ADAM2|chr8|-
NM_001465|FYB|chr5|-
NM_001462|FPR2|chr19|+
NM_001463|FRZB|chr2|-
NM_001460|FMO2|chr1|+
NM_000331|SAA1|chr11|+
NM_001164245|C1orf27|chr1|+
NM_007101|SARDH|chr9|-
NM_000332|ATXN1|chr6|-
NM_007107|SSR3|chr3|-
NM_007106|UBL3|chr13|-
NM_000337|SGCD|chr5|+
NM_000336|SCNN1B|chr16|+
NM_007109|TCF19|chr6|+
NM_007108|TCEB2|chr16|-
NM_001012969|ADAL|chr15|+
NM_001012968|SPIN4|chrX|-
NM_001012967|DDX60L|chr4|-
NM_001012965|KLK6|chr19|-
NM_001012964|KLK6|chr19|-
NM_003824|FADD|chr11|+
NM_003826|NAPG

In [17]:
All_sample_coverage_weights = All_samples_sequencing_depths/np.mean(All_samples_sequencing_depths)
All_sample_coverage_weights

array([1.02384623, 0.97615377])

In [20]:
num_samples = len(All_Sample_files)

##Prepare output directory
d = os.path.dirname(output_directory)
if not os.path.exists(d):
    os.makedirs(d)
temp_dir = d+'/tmp/'
if not os.path.exists(temp_dir):
    os.makedirs(temp_dir)
Output_all_prediction_file = output_directory+Output_result_file+'_result_temp.txt'
Output_result = open(Output_all_prediction_file, 'w')
##Write the first line
first_line = ['Gene','fit_value','Predicted_Proximal_APA','Loci']
for i in range(num_group_1):
    curr_long_exp = 'A_%s_long_exp' % str(i+1)
    curr_short_exp = 'A_%s_short_exp' % str(i+1)
    curr_ratio ='A_%s_PDUI' % str(i+1)
    first_line.extend([curr_long_exp,curr_short_exp,curr_ratio])
for i in range(num_samples - num_group_1):
    curr_long_exp = 'B_%s_long_exp' % str(i+1)
    curr_short_exp = 'B_%s_short_exp' % str(i+1)
    curr_ratio ='B_%s_PDUI' % str(i+1)
    first_line.extend([curr_long_exp,curr_short_exp,curr_ratio])
first_line.append('PDUI_Group_diff')

Output_result.writelines('\t'.join(first_line) + '\n')

In [35]:
    
for i,curr_3UTR_id in enumerate(UTR_events_dict):
    curr_3UTR_structure = UTR_events_dict[curr_3UTR_id]
    region_start = curr_3UTR_structure[1]
    region_end   = curr_3UTR_structure[2]
    curr_strand  = curr_3UTR_structure[-2]
    UTR_pos = curr_3UTR_structure[-1]
    if i==2:
        break

In [36]:
curr_3UTR_id

'NM_001008781|FAT3|chr11|+'

In [37]:
curr_3UTR_structure

['chr11', 92623658, 92628438, '+', 'chr11:92623657-92629635']

In [38]:
UTR_pos = curr_3UTR_structure[-1]
UTR_pos 

'chr11:92623657-92629635'

In [39]:
if curr_3UTR_id in All_samples_Target_3UTR_coverages:
    print(1)
    # curr_3UTR_coverage_wig = All_samples_Target_3UTR_coverages[curr_3UTR_id]
# curr_3UTR_coverage_wig